# 텐서 (PyTorch 버전)

In [1]:
# Environment & device
import torch
import numpy as np
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('torch', torch.__version__, '| cuda:', torch.cuda.is_available())

torch 2.8.0+cu128 | cuda: True


## 파이토치의 텐서

PyTorch는 단일 텐서 자료형(torch.Tensor)을 사용합니다. 불변/가변을 나누지 않지만, 자동 미분을 위해 requires_grad 플래그를 둘 수 있고, in-place 연산 시 주의가 필요합니다.

In [2]:
# 기본 생성
x = torch.tensor([[1., 2.], [3., 4.]])
print(x, x.dtype, x.shape)

tensor([[1., 2.],
        [3., 4.]]) torch.float32 torch.Size([2, 2])


In [3]:
# ones/zeros
x1 = torch.ones((2, 1))
print(x1)

tensor([[1.],
        [1.]])


In [4]:
x0 = torch.zeros((2, 1))
print(x0)

tensor([[0.],
        [0.]])


In [5]:
# 난수 텐서: 정규분포
xn = torch.randn((3, 1), dtype=torch.float32)  # mean=0, std=1
print(xn)

tensor([[ 0.2075],
        [-1.3101],
        [ 0.5118]])


In [6]:
# 균등분포 [0,1) -> [a,b): a + (b-a)*torch.rand(...)
xa = 0.0; xb = 1.0
xu = xa + (xb - xa) * torch.rand((3, 1), dtype=torch.float32)
print(xu)

tensor([[0.5738],
        [0.2653],
        [0.7870]])


### 텐서 수정 가능성(PyTorch)
PyTorch의 torch.Tensor는 기본적으로 가변(mutable)입니다. 인덱싱으로 항목을 직접 수정할 수 있습니다.

In [7]:
z = torch.ones((2, 2))
z[0, 0] = 0.0
print(z)

tensor([[0., 1.],
        [1., 1.]])


### 가변 텐서 대체(copy_)
모양이 같은 다른 텐서로 값을 통째로 바꿀 때는 `copy_()`를 사용합니다. (동일 shape 필요)

In [8]:
v = torch.randn((3, 1))
v.copy_(torch.ones((3, 1)))  # 동일 shape OK
print(v)

tensor([[1.],
        [1.],
        [1.]])


In [9]:
try:
    v.copy_(torch.ones((3, 2)))  # shape 불일치
except RuntimeError as e:
    print("RuntimeError:", e)

RuntimeError: output with shape [3, 1] doesn't match the broadcast shape [3, 2]


### 항목 수정과 in-place 연산
인덱싱으로 항목 수정 가능하며, `add_`, `sub_` 등 밑줄이 붙은 메서드는 in-place 연산입니다.

In [10]:
v = torch.zeros((3, 1))
v

tensor([[0.],
        [0.],
        [0.]])

In [11]:
v[0, 0] = 3.
print(v)

tensor([[3.],
        [0.],
        [0.]])


In [12]:
v.add_(2.)
print(v)

tensor([[5.],
        [2.],
        [2.]])


In [13]:
v.sub_(1.)
print(v)

tensor([[4.],
        [1.],
        [1.]])


## 인덱싱과 슬라이싱

In [14]:
t = torch.arange(1, 13).reshape(3, 4)
print(t)

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])


In [15]:
# 기본 인덱싱
print(t[0, 0])
print(t[1])       # 행 선택
print(t[:, 2])    # 열 선택

tensor(1)
tensor([5, 6, 7, 8])
tensor([ 3,  7, 11])


In [16]:
# 슬라이싱
print(t[:2, 1:3])

tensor([[2, 3],
        [6, 7]])


## 리셰이프, 전치

In [17]:
a = torch.arange(1, 13)
print(a.shape)

torch.Size([12])


In [18]:
b = a.reshape(3, 4)
print(b)
print(b.shape)

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])
torch.Size([3, 4])


In [19]:
# 2D 전치: T
print(b.T)
print(b.T.shape)

tensor([[ 1,  5,  9],
        [ 2,  6, 10],
        [ 3,  7, 11],
        [ 4,  8, 12]])
torch.Size([4, 3])


## 결합, 스택, 분할

In [20]:
a = torch.ones(2, 3)
b = torch.zeros(2, 3)
c = torch.cat([a, b], dim=0)
d = torch.cat([a, b], dim=1)

In [21]:
print(c)
print(c.shape)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]])
torch.Size([4, 3])


In [22]:
print(d)
print(d.shape)

tensor([[1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.]])
torch.Size([2, 6])


In [23]:
# stack: 새 축 추가
s = torch.stack([a, b], dim=0)
print(s)
print(s.shape)  # (2,2,3)

tensor([[[1., 1., 1.],
         [1., 1., 1.]],

        [[0., 0., 0.],
         [0., 0., 0.]]])
torch.Size([2, 2, 3])


In [24]:
# split/chunk
parts = torch.chunk(torch.arange(10), chunks=3)
print([p.shape for p in parts])

[torch.Size([4]), torch.Size([4]), torch.Size([2])]


In [25]:
parts2 = torch.split(torch.arange(10), [3, 3, 4])
print([p.shape for p in parts2])

[torch.Size([3]), torch.Size([3]), torch.Size([4])]


## 브로드캐스팅

In [26]:
x = torch.arange(3).reshape(3, 1)        # (3,1)
y = torch.arange(4).reshape(1, 4)        # (1,4)
print((x + y).shape)                      # (3,4)
print(x + y)

torch.Size([3, 4])
tensor([[0, 1, 2, 3],
        [1, 2, 3, 4],
        [2, 3, 4, 5]])


## 수학 연산, 행렬 곱, 축소 연산

In [27]:
x = torch.tensor([[1., 2.], [3., 4.]])
y = torch.tensor([[5., 6.], [7., 8.]])

In [28]:
# 원소별 연산
print(x + y)
print(x * y)

tensor([[ 6.,  8.],
        [10., 12.]])
tensor([[ 5., 12.],
        [21., 32.]])


In [29]:
# 행렬 곱
print(x @ y)           # 또는 torch.matmul(x, y)

tensor([[19., 22.],
        [43., 50.]])


In [30]:
# 축소 연산
m = torch.arange(1, 13).reshape(3, 4)
m

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])

In [31]:
print(m.sum())

tensor(78)


In [32]:
print(m.sum(dim=0))

tensor([15, 18, 21, 24])


In [33]:
print(m.sum(dim=1))

tensor([10, 26, 42])


In [34]:
print(m.to(torch.float32).mean(dim=1))

tensor([ 2.5000,  6.5000, 10.5000])


In [35]:
print(m.to(torch.float32).mean(dim=1, keepdim=True))

tensor([[ 2.5000],
        [ 6.5000],
        [10.5000]])


In [36]:
print(m.max(dim=1))     # 값과 인덱스 반환

torch.return_types.max(
values=tensor([ 4,  8, 12]),
indices=tensor([3, 3, 3]))


In [37]:
print(m.argmax(dim=1))

tensor([3, 3, 3])


## dtype

In [38]:
x = torch.randn(2, 2)
print(x.dtype)

torch.float32


In [39]:
x64 = x.to(torch.float64)
print(x64.dtype)

torch.float64


In [40]:
# 단축형 타입 캐스팅
print(x.float().dtype, x.long().dtype)

torch.float32 torch.int64


## 디바이스 이동

In [41]:
# 디바이스 이동
if torch.cuda.is_available():
    x_cuda = x.to('cuda')
    print(x_cuda.device)
    x_back = x_cuda.to('cpu')
    print(x_back.device)

cuda:0
cpu


## NumPy와 상호 호환성

In [42]:
# NumPy -> Torch (공유 메모리)
arr = np.array([1.0, 2.0, 3.0], dtype=np.float32)
t_from_np = torch.from_numpy(arr)
arr[0] = 10.0
print(t_from_np)  # 10.0 반영

tensor([10.,  2.,  3.])


In [43]:
# Torch -> NumPy (CPU 필요)
t_cpu = torch.tensor([4.0, 5.0, 6.0])
np_from_t = t_cpu.numpy()
np_from_t[1] = 50.0
print(t_cpu)  # 50.0 반영

tensor([ 4., 50.,  6.])


In [44]:
# GPU 텐서를 NumPy로 바꾸려면 CPU로 이동 필요
if torch.cuda.is_available():
    t_gpu = torch.tensor([1.0, 2.0, 3.0], device='cuda')
    print(t_gpu.to('cpu').numpy())

[1. 2. 3.]


## 시드와 재현성

In [45]:
torch.manual_seed(42)
print(torch.randn(3))

# Generator 사용
g = torch.Generator().manual_seed(123)
print(torch.randn(3, generator=g))
print(torch.randn(3, generator=g))  # 같은 생성기 재사용 시 다른 값

g2 = torch.Generator().manual_seed(123)
print(torch.randn(3, generator=g2))  # 같은 시드의 새 생성기 -> 처음과 동일

tensor([0.3367, 0.1288, 0.2345])
tensor([-0.1115,  0.1204, -0.3696])
tensor([-0.2404, -1.1969,  0.2093])
tensor([-0.1115,  0.1204, -0.3696])


## autograd 기본

In [46]:
x = torch.tensor([2.0, -3.0, 4.0], requires_grad=True)
y = (x ** 2).sum()
y.backward()
print(x.grad)

tensor([ 4., -6.,  8.])


In [47]:
# no_grad 문맥: 그래디언트 추적 비활성화
with torch.no_grad():
    z = x * 2.0
print(z.requires_grad)

False


In [48]:
# detach: 그래프에서 분리된 텐서 얻기
z2 = (x * 3.0).detach()
print(z2.requires_grad)

False


## 소프트맥스(Softmax)

- 분류 문제에서 계산된 값들을 확률값들의 분포로 변환합니다.

In [49]:
import torch
import torch.nn.functional as F

# 예제 1) 1D 텐서 (클래스 5개 대상)
tensor_1d = torch.tensor([2.0, 1.0, 0.1, -1.0, 3.5])
probs_1d = F.softmax(tensor_1d, dim=0)  # 1D는 dim=0이 유일 차원

print("[1D] probs:", probs_1d)
print("[1D] sum=", probs_1d.sum().item())  # 1.0

[1D] probs: tensor([0.1653, 0.0608, 0.0247, 0.0082, 0.7409])
[1D] sum= 1.0


In [50]:
# 예제 2) 2D 텐서 (배치 3, 클래스 4), 보통 dim=1(클래스 차원)
tensor_2d = torch.tensor([[2.0, 1.0, 0.1, -1.0],
                          [0.0, 0.0, 0.0, 0.0],
                          [3.0, 1.0, -2.0, 0.5]])

probs_2d = torch.softmax(tensor_2d, dim=1)  # F.softmax와 동일 동작

print("\n[2D] probs:\n", probs_2d)
print("[2D] row-wise sums:", probs_2d.sum(dim=1))  # 각 행 합=1


[2D] probs:
 tensor([[0.6381, 0.2347, 0.0954, 0.0318],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.8169, 0.1106, 0.0055, 0.0671]])
[2D] row-wise sums: tensor([1.0000, 1.0000, 1.0000])


In [51]:
# 로그 소프트맥스 (로그 확률)
log_probs = F.log_softmax(tensor_2d, dim=1)

print("\n[2D] log_probs:\n", log_probs)
print("\n[log_softmax] shape:", log_probs.shape)


[2D] log_probs:
 tensor([[-0.4493, -1.4493, -2.3493, -3.4493],
        [-1.3863, -1.3863, -1.3863, -1.3863],
        [-0.2023, -2.2023, -5.2023, -2.7023]])

[log_softmax] shape: torch.Size([3, 4])
